<a href="https://colab.research.google.com/github/carolvieirav/MaeTechDataRepo/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_de_Usu%C3%A1rias_M%C3%A3eTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [5]:
COLUNAS_EMPRESAS = [
    'Nome da Empresa',
    'Cargo',
    'Flexibilidade',
    'Benefícios',
    'Salário',
    'PJ ou CLT',
    'Tecnologias',
    'Tempo de experiência'
    'Area de experiência',
    'Escolaridade',
    'Escritório ou HomeOffice'
]

empresasdf = pd.DataFrame(columns=COLUNAS_EMPRESAS)

In [7]:
COLUNAS_USUARIAS = [
    'Nome Completo',
    'Cargo',
    'Flexibilidade',
    'Benefícios',
    'Salário',
    'PJ ou CLT',
    'Tecnologias',
    'Tempo de experiência'
    'Area de experiência',
    'Escolaridade',
    'Escritório ou HomeOffice'
]

usuariasdf = pd.DataFrame(columns=COLUNAS_USUARIAS)